# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 12:00AM,239501,10,MET211210,"Methapharm, Inc.",Released
1,Jun 14 2022 12:00AM,239505,10,0085759516,ISDIN Corporation,Released
2,Jun 14 2022 12:00AM,239505,10,0085759767,ISDIN Corporation,Released
3,Jun 14 2022 12:00AM,239505,10,0085759768,ISDIN Corporation,Released
4,Jun 14 2022 12:00AM,239514,10,0085759801,ISDIN Corporation,Released
5,Jun 14 2022 12:00AM,239514,10,0085759799,ISDIN Corporation,Released
6,Jun 14 2022 12:00AM,239514,10,0085759800,ISDIN Corporation,Released
7,Jun 14 2022 12:00AM,239505,10,0085759804,ISDIN Corporation,Released
8,Jun 14 2022 12:00AM,239514,10,0085759805,ISDIN Corporation,Released
9,Jun 14 2022 12:00AM,239505,10,0085759514,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,239501,Released,1
39,239504,Released,1
40,239505,Released,14
41,239514,Released,45
42,239517,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239501,NaN,NaN,1.0
239504,NaN,NaN,1.0
239505,NaN,NaN,14.0
239514,NaN,NaN,45.0
239517,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0.0,0.0,1.0
239389,0.0,0.0,1.0
239398,8.0,0.0,6.0
239407,0.0,1.0,0.0
239454,1.0,6.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239388,0,0,1
239389,0,0,1
239398,8,0,6
239407,0,1,0
239454,1,6,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,0,0,1
1,239389,0,0,1
2,239398,8,0,6
3,239407,0,1,0
4,239454,1,6,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239388,,,1
1,239389,,,1
2,239398,8,,6
3,239407,,1,
4,239454,1,6,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc."
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation
4,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation
60,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.
61,Jun 13 2022 3:38PM,239504,16,American International Chemical
62,Jun 13 2022 3:27PM,239498,10,"Methapharm, Inc."
65,Jun 13 2022 3:24PM,239497,22,"NBTY Global, Inc."
66,Jun 13 2022 3:11PM,239495,10,Bio-PRF
76,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc."
77,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc.",,,1
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation,,,14
2,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation,,,45
3,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.,,,1
4,Jun 13 2022 3:38PM,239504,16,American International Chemical,,,1
5,Jun 13 2022 3:27PM,239498,10,"Methapharm, Inc.",,,3
6,Jun 13 2022 3:24PM,239497,22,"NBTY Global, Inc.",,,1
7,Jun 13 2022 3:11PM,239495,10,Bio-PRF,,,10
8,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",,1,
9,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc.",1,,
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation,14,,
2,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation,45,,
3,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.,1,,
4,Jun 13 2022 3:38PM,239504,16,American International Chemical,1,,
5,Jun 13 2022 3:27PM,239498,10,"Methapharm, Inc.",3,,
6,Jun 13 2022 3:24PM,239497,22,"NBTY Global, Inc.",1,,
7,Jun 13 2022 3:11PM,239495,10,Bio-PRF,10,,
8,Jun 13 2022 2:53PM,239492,10,"Citieffe, Inc.",,1,
9,Jun 13 2022 2:49PM,239490,19,"NAPP Technologies, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc.",1,,
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation,14,,
2,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation,45,,
3,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.,1,,
4,Jun 13 2022 3:38PM,239504,16,American International Chemical,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation,14.0,NaN,NaN
2,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation,45.0,NaN,NaN
3,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.,1.0,NaN,NaN
4,Jun 13 2022 3:38PM,239504,16,American International Chemical,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 12:00AM,239501,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Jun 14 2022 12:00AM,239505,10,ISDIN Corporation,14.0,0.0,0.0
2,Jun 14 2022 12:00AM,239514,10,ISDIN Corporation,45.0,0.0,0.0
3,Jun 14 2022 12:00AM,239517,10,Eywa Pharma Inc.,1.0,0.0,0.0
4,Jun 13 2022 3:38PM,239504,16,American International Chemical,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3113366,141.0,15.0,48.0
12,478938,0.0,2.0,0.0
16,478911,1.0,1.0,0.0
18,239477,1.0,0.0,0.0
19,957913,3.0,20.0,0.0
20,957797,39.0,19.0,36.0
21,718448,3.0,0.0,0.0
22,718274,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3113366,141.0,15.0,48.0
1,12,478938,0.0,2.0,0.0
2,16,478911,1.0,1.0,0.0
3,18,239477,1.0,0.0,0.0
4,19,957913,3.0,20.0,0.0
5,20,957797,39.0,19.0,36.0
6,21,718448,3.0,0.0,0.0
7,22,718274,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,141.0,15.0,48.0
1,12,0.0,2.0,0.0
2,16,1.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,3.0,20.0,0.0
5,20,39.0,19.0,36.0
6,21,3.0,0.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,141.0
1,12,Released,0.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Completed,48.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0
Executing,15.0,2.0,1.0,0.0,20.0,19.0,0.0,0.0
Released,141.0,0.0,1.0,1.0,3.0,39.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Completed,48.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0
1,Executing,15.0,2.0,1.0,0.0,20.0,19.0,0.0,0.0
2,Released,141.0,0.0,1.0,1.0,3.0,39.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Completed,48.0,0.0,0.0,0.0,0.0,36.0,0.0,0.0
1,Executing,15.0,2.0,1.0,0.0,20.0,19.0,0.0,0.0
2,Released,141.0,0.0,1.0,1.0,3.0,39.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()